In [6]:
from mlat.dim2 import SquareABLattice


ml2d = SquareABLattice(k=1., m=[1, 1], precision=0.1)
ws, evecs = ml2d.dispersion()

# Dispersion relation

In [8]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Surface(z=ws[:, :, i], x=ml2d.qxs, y=ml2d.qys) for i in [2, 3]])
fig.show()

# Animation

In [9]:
import numpy as np
import plotly.graph_objects as go


# Parameters
qx = np.pi / 2
qy = - np.pi / 4
idx = min(range(len(ml2d.qxs)), key=lambda i: abs(ml2d.qxs[i] - qx))
idy = min(range(len(ml2d.qys)), key=lambda i: abs(ml2d.qys[i] - qy))
Q1 = np.exp(1.j * (qx - qy) / np.sqrt(2)) # SE direction
Q2 = np.exp(1.j * (qx + qy) / np.sqrt(2)) # NE direction
Q3 = np.exp(1.j * qx) # E direction

Nx = 10
Ny = 10
mode = 0 # 0 to 3
w = ws[idy, idx, mode] # /s

grid_cell_size = 2

grid = np.array([[[grid_cell_size * i, grid_cell_size * j] for i in range(Nx)] for j in range(Ny)], dtype=np.complex128)

# Print infos
print("Angular frequency: ", w)
print("Wave number: \nqx: ", qx, "\nqy: ", qy)

# Parameters for animation
fps = 30 # /s
animation_length = 3 # s

def generate_dgrid(shape, evecs_A, evecs_B):
    dgrid = np.zeros(shape, dtype=np.complex128)

    for i, x in enumerate(range(0, Nx, 2)):
        y = 0
        dgrid[y, x] = evecs_A * (Q3 ** i)
    for i, x in enumerate(range(1, Nx, 2)):
        y = 0
        dgrid[y, x] = evecs_B * (Q3 ** i)

    for y in range(1, Ny):
        for x in range(0, Nx, 2):
            dgrid[y, x] = dgrid[y - 1, x + 1] * Q2.conj()
            dgrid[y, x + 1] = dgrid[y - 1, x] * Q1
    return dgrid

evecs_A = evecs[idy, idx, mode, :2]
evecs_B = evecs[idy, idx, mode, 2:]
dgrid = generate_dgrid(grid.shape, evecs_A, evecs_B)

# Construct frames
frames = []
for t in range(int(animation_length * fps)):
    dt = t / fps
    dphase = dt * w * 2 * np.pi
    current_grid = grid + dgrid * np.exp(1.j * dphase)
    x = current_grid[:, :, 0].flatten()
    y = current_grid[:, :, 1].flatten()
    frames.append(go.Frame(data=[go.Scatter(x=x.real, y=y.real, mode='markers')]))

# Figure components
start_button = dict(
        label="Play",
        method="animate",
        args=[
            None, 
            {
                "frame":{"duration": 1000 / fps, "redraw": False},
                "fromcurrent": True, 
                "transition": {"duration": 100}
            }])
pause_button = dict(
        label="Pause",
        method="animate",
        args=[
            [None], 
            {
                "frame": {"duration": 0, "redraw": False},
                "mode": "immediate",
                "transition": {"duration": 0}
            }])

# Plot
fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title="Dispersion relation animation",
        yaxis=dict(scaleanchor="x", scaleratio=1, autorange='reversed'),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[start_button, pause_button
            ])
        ]
    ),
    frames=frames[1:])
fig.show()

Angular frequency:  0.5183976722365841
Wave number: 
qx:  1.5707963267948966 
qy:  -0.7853981633974483
